In [1]:
import pandas as pd
import requests
import csv
import json
from collections import OrderedDict
from heapq import nlargest
from heapq import nsmallest

### Collect and filter dataset for useful values and assign column names

In [2]:
data = pd.read_excel("DDW-0000C-14.xlsx", sheet_name='Sheet1')
data.head(7)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,C-14 POPULATION IN FIVE YEAR AGE-GROUP BY RESIDENCE AND SEX,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,Table,State,Distt.,Area Name,Age-group,Total,NaN,NaN,Rural,NaN,NaN,Urban,NaN,NaN
1,Name,Code,Code,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,Persons,Males,Females,Persons,Males,Females,Persons,Males,Females
3,NaN,NaN,NaN,NaN,1,2,3,4,5,6,7,8,9,10
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,C4114,00,000,India,All ages,1210854977,623270258,587584719,833748852,427781058,405967794,377106125,195489200,181616925


In [3]:
data = data.loc[6:,:]
data.drop('Unnamed: 0',axis='columns', inplace=True)
data

,Unnamed: 1,Unnamed: 2,Unnamed: 3,C-14 POPULATION IN FIVE YEAR AGE-GROUP BY RESIDENCE AND SEX,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
6,00,000,India,All ages,1210854977,623270258,587584719,833748852,427781058,405967794,377106125,195489200,181616925
7,00,000,India,0-4,112806778,58632074,54174704,82986660,43036377,39950283,29820118,15595697,14224421
8,00,000,India,5-9,126928126,66300466,60627660,93807612,48825259,44982353,33120514,17475207,15645307
9,00,000,India,10-14,132709212,69418835,63290377,96804494,50488158,46316336,35904718,18930677,16974041
10,00,000,India,15-19,120526449,63982396,56544053,83902472,44570557,39331915,36623977,19411839,17212138
...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,35,000,ANDAMAN & NICOBAR ISLANDS,65-69,6226,3430,2796,4469,2477,1992,1757,953,804
686,35,000,ANDAMAN & NICOBAR ISLANDS,70-74,4098,2378,1720,2905,1687,1218,1193,691,502
687,35,000,ANDAMAN & NICOBAR ISLANDS,75-79,2251,1295,956,1606,952,654,645,343,302
688,35,000,ANDAMAN & NICOBAR ISLANDS,80+,2473,1279,1194,1857,992,865,616,287,329


In [4]:
['State_Code','District_Code','Area_Name','Age-group','Total_Persons','Total_Males','Total_Females','Rural_Persons','Rural_Males','Rural_Females','Urban_Persons','Urban_Males','Urban_Females']

['State_Code',
 'District_Code',
 'Area_Name',
 'Age-group',
 'Total_Persons',
 'Total_Males',
 'Total_Females',
 'Rural_Persons',
 'Rural_Males',
 'Rural_Females',
 'Urban_Persons',
 'Urban_Males',
 'Urban_Females']

In [5]:
agegroup_data = data.set_axis(['State_Code','District_Code','Area_Name','Age-group','Total_Persons','Total_Males','Total_Females','Rural_Persons','Rural_Males','Rural_Females','Urban_Persons','Urban_Males','Urban_Females'], axis=1, inplace=False)
agegroup_data

,State_Code,District_Code,Area_Name,Age-group,Total_Persons,Total_Males,Total_Females,Rural_Persons,Rural_Males,Rural_Females,Urban_Persons,Urban_Males,Urban_Females
6,00,000,India,All ages,1210854977,623270258,587584719,833748852,427781058,405967794,377106125,195489200,181616925
7,00,000,India,0-4,112806778,58632074,54174704,82986660,43036377,39950283,29820118,15595697,14224421
8,00,000,India,5-9,126928126,66300466,60627660,93807612,48825259,44982353,33120514,17475207,15645307
9,00,000,India,10-14,132709212,69418835,63290377,96804494,50488158,46316336,35904718,18930677,16974041
10,00,000,India,15-19,120526449,63982396,56544053,83902472,44570557,39331915,36623977,19411839,17212138
...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,35,000,ANDAMAN & NICOBAR ISLANDS,65-69,6226,3430,2796,4469,2477,1992,1757,953,804
686,35,000,ANDAMAN & NICOBAR ISLANDS,70-74,4098,2378,1720,2905,1687,1218,1193,691,502
687,35,000,ANDAMAN & NICOBAR ISLANDS,75-79,2251,1295,956,1606,952,654,645,343,302
688,35,000,ANDAMAN & NICOBAR ISLANDS,80+,2473,1279,1194,1857,992,865,616,287,329


In [6]:
df1 = agegroup_data.groupby(['Area_Name'])
df1.first()

,State_Code,District_Code,Age-group,Total_Persons,Total_Males,Total_Females,Rural_Persons,Rural_Males,Rural_Females,Urban_Persons,Urban_Males,Urban_Females
Area_Name,,,,,,,,,,,,
ANDAMAN & NICOBAR ISLANDS,35,000,All ages,380581,202871,177710,237093,126287,110806,143488,76584,66904
ANDHRA PRADESH,28,000,All ages,84580777,42442146,42138631,56361702,28243241,28118461,28219075,14198905,14020170
ARUNACHAL PRADESH,12,000,All ages,1383727,713912,669815,1066358,546011,520347,317369,167901,149468
ASSAM,18,000,All ages,31205576,15939443,15266133,26807034,13678989,13128045,4398542,2260454,2138088
BIHAR,10,000,All ages,104099452,54278157,49821295,92341436,48073850,44267586,11758016,6204307,5553709
CHANDIGARH,04,000,All ages,1055450,580663,474787,28991,17150,11841,1026459,563513,462946
CHHATTISGARH,22,000,All ages,25545198,12832895,12712303,19607961,9797426,9810535,5937237,3035469,2901768
DADRA & NAGAR HAVELI,26,000,All ages,343709,193760,149949,183114,98305,84809,160595,95455,65140
DAMAN & DIU,25,000,All ages,243247,150301,92946,60396,32395,28001,182851,117906,64945


In [7]:
groups_name = list(df1.groups.keys())
groups_name

['ANDAMAN & NICOBAR ISLANDS',
 'ANDHRA PRADESH',
 'ARUNACHAL PRADESH',
 'ASSAM',
 'BIHAR',
 'CHANDIGARH',
 'CHHATTISGARH',
 'DADRA & NAGAR HAVELI',
 'DAMAN & DIU',
 'GOA',
 'GUJARAT',
 'HARYANA',
 'HIMACHAL PRADESH',
 'India',
 'JAMMU & KASHMIR',
 'JHARKHAND',
 'KARNATAKA',
 'KERALA',
 'LAKSHADWEEP',
 'MADHYA PRADESH',
 'MAHARASHTRA',
 'MANIPUR',
 'MEGHALAYA',
 'MIZORAM',
 'NAGALAND',
 'NCT OF DELHI',
 'ODISHA',
 'PUDUCHERRY',
 'PUNJAB',
 'RAJASTHAN',
 'SIKKIM',
 'TAMIL NADU',
 'TRIPURA',
 'UTTAR PRADESH',
 'UTTARAKHAND',
 'WEST BENGAL']

In [8]:
language_data = pd.read_excel("DDW-C18-0000.xlsx", sheet_name='Sheet1')

In [9]:
language_data.head(7)

,"C-18 POPULATION BY BILINGUALISM, TRILINGUALISM, AGE AND SEX",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,State,District,Area Name,Total/,Age-group,Number speaking second language,NaN,NaN,Number speaking third language,NaN,NaN
1,code,code,NaN,Rural/,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,Urban,NaN,Persons,Males,Females,Persons,Males,Females
3,1,2,3,4,5,6,7,8,9,10,11
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,00,00,INDIA,Total,Total,314988770,176696383,138292387,86009580,50536832,35472748
6,00,00,INDIA,Total,5-9,15649192,8166843,7482349,1844108,978151,865957


In [10]:
language_data = language_data.loc[5:,:]

In [11]:
['State_Code','District_Code','Area_Name','Total/Rural/Urban','Age-group','Number_speaking_second_language','Males2','Females2','Number_speaking_third_language','Males3','Females3']

['State_Code',
 'District_Code',
 'Area_Name',
 'Total/Rural/Urban',
 'Age-group',
 'Number_speaking_second_language',
 'Males2',
 'Females2',
 'Number_speaking_third_language',
 'Males3',
 'Females3']

In [12]:
language_data_total = language_data.set_axis(['State_Code','District_Code','Area_Name','Total/Rural/Urban','Age-group','Number_speaking_second_language','Males2','Females2','Number_speaking_third_language','Males3','Females3'], axis=1, inplace=False)
language_data_total

,State_Code,District_Code,Area_Name,Total/Rural/Urban,Age-group,Number_speaking_second_language,Males2,Females2,Number_speaking_third_language,Males3,Females3
5,00,00,INDIA,Total,Total,314988770,176696383,138292387,86009580,50536832,35472748
6,00,00,INDIA,Total,5-9,15649192,8166843,7482349,1844108,978151,865957
7,00,00,INDIA,Total,10-14,34488492,18133423,16355069,7254335,3831131,3423204
8,00,00,INDIA,Total,15-19,42424599,22750908,19673691,12626717,6792766,5833951
9,00,00,INDIA,Total,20-24,41344406,22386694,18957712,12834334,7067614,5766720
...,...,...,...,...,...,...,...,...,...,...,...
1080,35,00,ANDAMAN & NICOBAR ISLANDS,Urban,25-29,13065,7012,6053,4364,2355,2009
1081,35,00,ANDAMAN & NICOBAR ISLANDS,Urban,30-49,38406,21301,17105,13320,7824,5496
1082,35,00,ANDAMAN & NICOBAR ISLANDS,Urban,50-69,12899,7903,4996,4177,2842,1335
1083,35,00,ANDAMAN & NICOBAR ISLANDS,Urban,70+,1644,1016,628,432,305,127


### Calculating State Level Data

In [13]:
df1 = language_data_total.groupby(['Area_Name'])
df1.first()

,State_Code,District_Code,Total/Rural/Urban,Age-group,Number_speaking_second_language,Males2,Females2,Number_speaking_third_language,Males3,Females3
Area_Name,,,,,,,,,,
ANDAMAN & NICOBAR ISLANDS,35,00,Total,Total,257425,142378,115047,72708,41846,30862
ANDHRA PRADESH,28,00,Total,Total,21583334,12193992,9389342,6344376,3836382,2507994
ARUNACHAL PRADESH,12,00,Total,Total,885945,478413,407532,418551,236666,181885
ASSAM,18,00,Total,Total,14460591,7979404,6481187,4352497,2636287,1716210
BIHAR,10,00,Total,Total,16761785,9443495,7318290,1682572,1028475,654097
CHANDIGARH,04,00,Total,Total,579920,322898,257022,321979,178557,143422
CHHATTISGARH,22,00,Total,Total,3384223,1820621,1563602,422569,245150,177419
DADRA & NAGAR HAVELI,26,00,Total,Total,198983,113211,85772,71516,44256,27260
DAMAN & DIU,25,00,Total,Total,116629,71260,45369,34602,21856,12746


In [14]:
group_state = list(df1.groups.keys())
group_state

['ANDAMAN & NICOBAR ISLANDS',
 'ANDHRA PRADESH',
 'ARUNACHAL PRADESH',
 'ASSAM',
 'BIHAR',
 'CHANDIGARH',
 'CHHATTISGARH',
 'DADRA & NAGAR HAVELI',
 'DAMAN & DIU',
 'GOA',
 'GUJARAT',
 'HARYANA',
 'HIMACHAL PRADESH',
 'INDIA',
 'JAMMU & KASHMIR',
 'JHARKHAND',
 'KARNATAKA',
 'KERALA',
 'LAKSHADWEEP',
 'MADHYA PRADESH',
 'MAHARASHTRA',
 'MANIPUR',
 'MEGHALAYA',
 'MIZORAM',
 'NAGALAND',
 'NCT OF DELHI',
 'ODISHA',
 'PUDUCHERRY',
 'PUNJAB',
 'RAJASTHAN',
 'SIKKIM',
 'TAMIL NADU',
 'TRIPURA',
 'UTTAR PRADESH',
 'UTTARAKHAND',
 'WEST BENGAL']

In [15]:
df2 = agegroup_data.groupby(['Area_Name'])
df2.first()

,State_Code,District_Code,Age-group,Total_Persons,Total_Males,Total_Females,Rural_Persons,Rural_Males,Rural_Females,Urban_Persons,Urban_Males,Urban_Females
Area_Name,,,,,,,,,,,,
ANDAMAN & NICOBAR ISLANDS,35,000,All ages,380581,202871,177710,237093,126287,110806,143488,76584,66904
ANDHRA PRADESH,28,000,All ages,84580777,42442146,42138631,56361702,28243241,28118461,28219075,14198905,14020170
ARUNACHAL PRADESH,12,000,All ages,1383727,713912,669815,1066358,546011,520347,317369,167901,149468
ASSAM,18,000,All ages,31205576,15939443,15266133,26807034,13678989,13128045,4398542,2260454,2138088
BIHAR,10,000,All ages,104099452,54278157,49821295,92341436,48073850,44267586,11758016,6204307,5553709
CHANDIGARH,04,000,All ages,1055450,580663,474787,28991,17150,11841,1026459,563513,462946
CHHATTISGARH,22,000,All ages,25545198,12832895,12712303,19607961,9797426,9810535,5937237,3035469,2901768
DADRA & NAGAR HAVELI,26,000,All ages,343709,193760,149949,183114,98305,84809,160595,95455,65140
DAMAN & DIU,25,000,All ages,243247,150301,92946,60396,32395,28001,182851,117906,64945


### Calculating State Data

In [16]:
state_dict = dict()
country_dict = dict()

In [17]:
age_range = {'5-9':['5-9'],'10-14':['10-14'],'15-19':['15-19'],'20-24':['20-24'],'25-29':['25-29'],'30-49':['30-34','35-39','40-44','45-49'],'50-69':['50-54','55-59','60-64','65-69'],'70+':['70-74','75-79','80+']}

In [18]:
for i in range(len(group_state)):
    key = group_state[i]
    z = df1.get_group(group_state[i])          #retrieve the dataset under that state
    z = z[z['Total/Rural/Urban'] == 'Total']
    k = z.iloc[0]
    State = key
    status=0
    if key=='INDIA':
        status = 1
        key = 'India'
    #print("State = ",key)
    z2 = df2.get_group(key)
    highest_percentage = 0
    for key, value in age_range.items():
        age_gp = key
        row1 = z[z['Age-group'] == age_gp]
        Number_speaking_third_language = int(row1['Number_speaking_third_language'])
        population = 0
        for item in value: # Find valued according to age group in Language dataset 
            pop_key = item
            row2 = z2[z2['Age-group'] == pop_key]
            population = population + int(row2['Total_Persons'])
        percentage = (Number_speaking_third_language/population)*100
        if percentage>highest_percentage:
            Group = age_gp
            highest_percentage = percentage
    if status==0:
        #print("Group = ",Group)
        #print("highest_percentage = ",highest_percentage)
        state_dict[State] = [Group,round(highest_percentage,3)]
    else:
        #print("Group = ",Group)
        #print("highest_percentage = ",highest_percentage)
        country_dict[State] = [Group,round(highest_percentage,3)]
    #print("")

In [19]:
state_dict

{'ANDAMAN & NICOBAR ISLANDS': ['15-19', 28.747],
 'ANDHRA PRADESH': ['15-19', 12.981],
 'ARUNACHAL PRADESH': ['15-19', 47.589],
 'ASSAM': ['20-24', 21.066],
 'BIHAR': ['15-19', 2.918],
 'CHANDIGARH': ['50-69', 42.775],
 'CHHATTISGARH': ['25-29', 2.296],
 'DADRA & NAGAR HAVELI': ['15-19', 33.83],
 'DAMAN & DIU': ['30-49', 18.178],
 'GOA': ['15-19', 68.089],
 'GUJARAT': ['15-19', 23.987],
 'HARYANA': ['15-19', 7.319],
 'HIMACHAL PRADESH': ['20-24', 8.207],
 'JAMMU & KASHMIR': ['20-24', 29.58],
 'JHARKHAND': ['20-24', 7.375],
 'KARNATAKA': ['20-24', 17.577],
 'KERALA': ['20-24', 16.946],
 'LAKSHADWEEP': ['20-24', 25.991],
 'MADHYA PRADESH': ['15-19', 2.963],
 'MAHARASHTRA': ['20-24', 27.068],
 'MANIPUR': ['25-29', 31.663],
 'MEGHALAYA': ['30-49', 14.863],
 'MIZORAM': ['30-49', 7.42],
 'NAGALAND': ['20-24', 40.34],
 'NCT OF DELHI': ['70+', 14.999],
 'ODISHA': ['15-19', 22.31],
 'PUDUCHERRY': ['20-24', 7.852],
 'PUNJAB': ['15-19', 45.764],
 'RAJASTHAN': ['15-19', 2.884],
 'SIKKIM': ['15-19'

In [20]:
a_file = open("age-india-b.csv", "w",newline="")
writer = csv.DictWriter(a_file, fieldnames = ['state/ut', 'age-group', 'percentage'])
writer.writeheader()
writer = csv.writer(a_file)
for key, value in state_dict.items():
    writer.writerow([key] + value)

a_file.close()

In [21]:
data = pd.read_csv("age-india-b.csv")
data = data.to_string(index=False)
print("Displaying states and union territories of India data")
print("")
print(data)

Displaying states and union territories of India data

                 state/ut age-group  percentage
ANDAMAN & NICOBAR ISLANDS     15-19      28.747
           ANDHRA PRADESH     15-19      12.981
        ARUNACHAL PRADESH     15-19      47.589
                    ASSAM     20-24      21.066
                    BIHAR     15-19       2.918
               CHANDIGARH     50-69      42.775
             CHHATTISGARH     25-29       2.296
     DADRA & NAGAR HAVELI     15-19      33.830
              DAMAN & DIU     30-49      18.178
                      GOA     15-19      68.089
                  GUJARAT     15-19      23.987
                  HARYANA     15-19       7.319
         HIMACHAL PRADESH     20-24       8.207
          JAMMU & KASHMIR     20-24      29.580
                JHARKHAND     20-24       7.375
                KARNATAKA     20-24      17.577
                   KERALA     20-24      16.946
              LAKSHADWEEP     20-24      25.991
           MADHYA PRADESH     15-

In [22]:
print("")

In [23]:
country_dict

{'INDIA': ['20-24', 11.518]}

In [24]:
a_file = open("age-india-a.csv", "w",newline="")
writer = csv.DictWriter(a_file, fieldnames = ['Country', 'age-group', 'percentage'])
writer.writeheader()
writer = csv.writer(a_file)
for key, value in country_dict.items():
    writer.writerow([key] + value)

a_file.close()

In [25]:
data = pd.read_csv("age-india-a.csv")
data = data.to_string(index=False)
print("Displaying over all country data")
print("")
print(data)

Displaying over all country data

Country age-group  percentage
  INDIA     20-24      11.518
